In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sagemaker boto3 awscli

In [ ]:
!aws s3 ls --profile asher

In [ ]:
local_dir = "./data"

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
role = "arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996" # get_execution_role() #
print("Role: ", role)
# sess = LocalSession()  # for local session
# local_sess.config = {'local': {'local_code': True}}

sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1")) #name of sso session remove profile name in sagemaker studio , profile_name="asher"
# #for remote session
# settings = dict(
#     sagemaker_session=sess,
#     role=role,
#     instance_type="ml.t3.large",
#     dependencies='./requirements.txt',
# )

bucket_name = sess.default_bucket()
print(bucket_name)

Role:  arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996
sagemaker-ap-southeast-1-345594598345


Uploaded training, testing and validation datasets to S3 bucket

In [ ]:
# !aws s3 cp ./data s3://{bucket_name}/prostate158 --recursive --profile asher
# print("Done!")

Upload weights of pre-trained UNet model

In [ ]:
# !aws s3 cp models/tumor.pt s3://{bucket_name}/models/tumor.pt --profile asher

In [17]:
# !aws s3 cp tumor.yaml s3://{bucket_name}/configs/tumor.yaml

upload: ./tumor.yaml to s3://sagemaker-ap-southeast-1-345594598345/configs/tumor.yaml


In [18]:
if bucket_name:
    train_channel = f"s3://{bucket_name}/prostate158/prostate158_train/train"
    test_channel = f"s3://{bucket_name}/prostate158/prostate158_test/test"
    model_channel = f"s3://{bucket_name}/models/tumor.pt"
    tumor_config = f"s3://{bucket_name}/configs/tumor.yaml"
inputs = {"train": train_channel, "test": test_channel, "model": model_channel, "training_config": tumor_config}
print(inputs)

{'train': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_train/train', 'test': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_test/test', 'model': 's3://sagemaker-ap-southeast-1-345594598345/models/tumor.pt', 'training_config': 's3://sagemaker-ap-southeast-1-345594598345/configs/tumor.yaml'}


In [9]:
!pip install munch

  Using cached munch-4.0.0-py2.py3-none-any.whl.metadata (5.9 kB)
Using cached munch-4.0.0-py2.py3-none-any.whl (9.9 kB)


In [11]:
import yaml
from pathlib import Path
from munch import Munch
# config = yaml.safe_load(Path("tumor.yaml").read_text())
config = Munch.fromDict(yaml.safe_load(Path("tumor.yaml").read_text()))
config

Munch({'debug': False, 'overwrite': False, 'ndim': 3, 'run_id': 'tumor', 'out_dir': '/opt/ml/model/', 'model_dir': '/opt/ml/input/data/model/', 'log_dir': 'logs', 'seed': 42, 'device': 'cuda', 'num_workers': 4, 'data': Munch({'data_dir': '/opt/ml/input/data/', 'train_csv': './prostate158/train.csv', 'valid_csv': './prostate158/valid.csv', 'test_csv': './prostate158/test.csv', 'image_cols': ['t2', 'adc', 'dwi'], 'label_cols': 'adc_tumor_reader1', 'train': True, 'valid': True, 'test': False, 'dataset_type': 'persistent', 'cache_dir': '/tmp/monai-cache', 'batch_size': 1}), 'transforms': Munch({'prob': 0.175, 'spacing': [0.5, 0.5, 0.5], 'orientation': 'RAS', 'rand_bias_field': Munch({'degree': 2, 'coeff_range': [0.0, 0.01]}), 'rand_gaussian_smooth': Munch({'sigma_x': [0.25, 1.5], 'sigma_y': [0.25, 1.5], 'sigma_z': [0.25, 1.5]}), 'rand_gibbs_nose': Munch({'alpha': [0.5, 1]}), 'rand_affine': Munch({'rotate_range': 5, 'shear_range': 0.5, 'translate_range': 25}), 'rand_rotate90': Munch({'spati

In [ ]:
%%time
from sagemaker.pytorch.estimator import PyTorch
import os

image = "763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:2.7.1-gpu-py312-cu128-ubuntu22.04-sagemaker"

estimator = PyTorch(
    image_uri=image,
    entry_point="train_gpt.py",
    source_dir=".",
    role=role,
    region="ap-southeast-1",
    instance_type="ml.g4dn.2xlarge", # ml.g5.xlarge ran out of memory halfway through 2nd epoch (free tier instance)
    # instance_type='local',
    instance_count=1,
    framework_version="2.7.1",
    py_version="py312",
    enable_spot_training=True,  # for cost savings
    input_mode="File",
    sagemaker_session=sess,
    distribution={"torch_distributed": {"enabled": True}}, #use torchrun backend
    # distribution={"pytorchddp": {"enabled": True}}, #use mpirun backend
)
inputs={
    "training": train_channel,
    "models": model_channel,
    "training_config": os.environ.get("SM_INPUT_TRAINING_CONFIG", tumor_config),
}
estimator.fit(inputs)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-07-17-08-41-56-628


2025-07-17 08:42:00 Starting - Starting the training job...
2025-07-17 08:42:15 Starting - Preparing the instances for training...
2025-07-17 08:42:38 Downloading - Downloading input data...
2025-07-17 08:43:08 Downloading - Downloading the training image........................
2025-07-17 08:47:05 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
Skipping CUDA compat setup as package not found
2025-07-17 08:47:31,879 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-17 08:47:31,896 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-17 08:47:31,905 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-07-17 08:47:31,937 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-07-17 08:47:33,682 sagema

In [ ]:
# !aws s3 cp s3://sagemaker-ap-southeast-1-345594598345/pytorch-training-2025-07-15-08-01-51-392/output/model.tar.gz .
#!tar -xvzf model.tar.gz

import nibabel as nib
import boto3
import os

s3 = boto3.client('s3')

job_name = estimator.latest_training_job.name
model_artifact_s3_uri = estimator.model_data

print("Model artifact is at:", model_artifact_s3_uri)

destination_key = "models/model.tar.gz"

# Perform the copy
s3.copy_object(
    Bucket=bucket_name,  # Destination bucket
    CopySource={
        'Bucket': bucket_name,  # Source bucket
        'Key': model_artifact_s3_uri
    },
    Key=destination_key  # Destination key (what you want it to be)
)

print(f"Copied model to s3://{bucket_name}/{destination_key}")


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
network_tumor_1_key_metric=0.0066.pt
